### 랭체인 준비

In [ ]:
# 패키지 설치
!pip install langchain==0.3.7
!pip install langchain-google-genai
!pip install langchain_community
!pip install langgraph

In [ ]:
import os
from google.colab import userdata

# 환경 변수 준비(좌측 상단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

### 문자열 출력 파서 사용법

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# 프롬프트 템플릿 준비
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "다음 질문에 답변해주세요. 답변만 남겨주세요."),
    ("human", "{question}"),
])

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 문자열 출력 파서 준비
parser = StrOutputParser()

In [ ]:
# 체인 준비
chain = prompt_template | llm | parser

# 체인 실행
output = chain.invoke({"question": "대규모 언어 모델 Gemini가 발표된 연도와 날짜는 언제인가요?"})
print(type(output))
print(output)

<class 'str'>
2023년 12월 6일 



### 단순 JSON 출력 파서 사용법

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# 프롬프트 템플릿 준비
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "다음 질문에 답변해주세요. JSON형식으로 반환해 주세요."),
    ("human", "{question}"),
])

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

# 단순 JSON 출력 파서 준비
parser = SimpleJsonOutputParser()

In [ ]:
# 체인 준비
chain = prompt_template | llm | parser

# 체인 실행
output = chain.invoke({"question": "대규모 언어 모델 Gemini가 발표된 연도와 날짜는 언제인가요?"})
print(type(output))
print(output)

<class 'dict'>
{'발표 연도': '2023', '발표 날짜': '12월 6일'}


### 파이단틱 출력 파서 사용법

In [ ]:
from pydantic import BaseModel, Field

# 구조화 데이터 정의
class Ym(BaseModel):
    year: int = Field(description="year (e.g. 2001)")
    month: int = Field(description="month (e.g. 1)")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

# 파이단틱 출력 파서 준비
parser = PydanticOutputParser(pydantic_object=Ym)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# 프롬프트 템플릿 준비
prompt_template = PromptTemplate.from_template(
    template="다음 질문에 답변해주세요.\n{format_instructions}\n{question}",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# LLM 준비
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
)

In [ ]:
# 지시 확인
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"year": {"description": "year (e.g. 2001)", "title": "Year", "type": "integer"}, "month": {"description": "month (e.g. 1)", "title": "Month", "type": "integer"}}, "required": ["year", "month"]}\n```'

In [ ]:
# 체인 준비
chain = prompt_template | llm | parser

# 체인 실행
output = chain.invoke({"question": "대규모 언어 모델 Gemini가 발표된 연도와 날짜는 언제인가요?"})
print(type(output))
print(output)

<class '__main__.Ym'>
year=2022 month=12
